In [127]:
''' import '''
import os
import numpy as np
import matplotlib.pyplot as plt

import pydicom
import nibabel as nib

from dicom import dcmread_volume

In [128]:
root_path = os.path.join(os.getcwd(), '..', '..',  'dicom_matlab', 'data', 'patient-example')

ct_folder = None
rtdose_folder = None
rtst_folder = None

ct_files = []
rtdose_files = []
rtst_files = []

# folders
for folder in os.listdir(root_path):
    if 'CT' in folder:
        ct_folder = os.path.join(root_path, folder)
    if 'RTDOSE' in folder:
        rtdose_folder = os.path.join(root_path, folder)
    if 'RTst' in folder:
        rtst_folder = os.path.join(root_path, folder)

# files
for file in os.listdir(ct_folder): # CT
    if file.endswith('.dcm'):
        ct_files.append(os.path.join(ct_folder, file))

for file in os.listdir(rtdose_folder): # RT dose
    if file.endswith('.dcm'):
        rtdose_files.append(os.path.join(rtdose_folder, file))

for file in os.listdir(rtdose_folder): # RT structure
    if file.endswith('.dcm'):
        rtst_files.append(os.path.join(rtst_folder, file))

# file
rtdose_file = rtdose_files[0]
rtst_file = rtst_files[0]

# CT

In [134]:
img_origin = np.zeros(3, dtype=np.double)
img_spacing = np.zeros(3, dtype=np.double)
img_size = np.zeros(3, dtype=np.int16)


''' CT meta '''
ct_info_tmp = pydicom.dcmread(ct_files[0])

rescale_slope = ct_info_tmp.RescaleSlope
rescale_intercept = ct_info_tmp.RescaleIntercept

slice_thickness = ct_info_tmp.SliceThickness


''' CT img '''
image_raw, spatial = dcmread_volume(ct_folder)

img_origin[:] = spatial.PatientPositions[0, :]
img_spacing[:2] = spatial.PixelSpacings[0, :]
img_spacing[2] = slice_thickness
img_size[:] = spatial.ImageSize # 512 512 337 (이건 웃기게도 python, matlab 동일)

image = (image_raw*rescale_slope) + rescale_intercept
image = np.transpose(image, (1,0,2)) # y x z -> x y z

In [130]:
''' save nii '''
ct_affine = np.array([
    [img_spacing[0], 0, 0, img_origin[0]],
    [0, img_spacing[1], 0, img_origin[1]],
    [0, 0, img_spacing[2], img_origin[2]],
    [0, 0, 0, 1]
])

ct_nii = nib.Nifti1Image(image, ct_affine)

# nib.save(ct_nii, '../data/ct.nii')

In [ ]:
''' read nii '''
ct_data = nib.load('../data/ct.nii')

ct_info = ct_data.header
ct_affine = ct_data.affine
image = ct_data.get_fdata()

print(ct_info)

In [ ]:
''' plot nii '''
img_origin = ct_affine[:3,3]
img_size = ct_info['dim'][1:4]
img_spacing = ct_info.get_zooms()

img_x = np.linspace(img_origin[0], img_origin[0] + img_spacing[0]*(img_size[0] - 1), img_size[0])
img_y = np.linspace(img_origin[1], img_origin[1] + img_spacing[1]*(img_size[1] - 1), img_size[1])
img_z = np.linspace(img_origin[2], img_origin[2] + img_spacing[2]*(img_size[2] - 1), img_size[2])

_, axs = plt.subplots(1,3, figsize=(15,5))

axs[0].set_title('axial')
axs[0].imshow(image[:,:,160], cmap='gray', 
              extent=[img_y[0], img_y[-1], img_x[-1], img_x[0]])

axs[1].set_title('coronal')
axs[1].imshow(image[250,:,:].T, cmap='gray',
              extent=[img_y[0], img_y[-1], img_z[-1], img_z[0]])
axs[1].invert_yaxis()

axs[2].set_title('sagittal')
axs[2].imshow(image[:,250,:].T, cmap='gray', 
              extent=[img_x[0], img_x[-1], img_z[-1], img_z[0]])
axs[2].invert_yaxis()

plt.show()

# RT dose

In [123]:
rtdose_origin = np.zeros(3, dtype=np.double)
rtdose_spacing = np.zeros(3, dtype=np.double)
rtdose_size = np.zeros(3, dtype=np.int16)


''' RT dose meta'''
rtdose_info = pydicom.dcmread(rtdose_file)

grid_scale = rtdose_info.DoseGridScaling
slice_thickness = rtdose_info.SliceThickness


''' RT dose img '''
rtdose_raw = rtdose_info.pixel_array

rtdose_origin[:] = rtdose_info.ImagePositionPatient
rtdose_spacing[:2] = rtdose_info.PixelSpacing
rtdose_spacing[2] = slice_thickness
rtdose_size[:] = rtdose_raw.shape   # python(z y x) : 317 143 267   / matlab(y x z) : 143 267 317 ----> dcm/nii(x y z)

rtdose = rtdose_raw*grid_scale
rtdose = np.transpose(rtdose, (2,1,0)) # z y x -> x y z

In [ ]:
rtdose_x = np.linspace(rtdose_origin[0], rtdose_origin[0] + rtdose_spacing[0]*(rtdose_size[0] - 1), rtdose_size[0])
rtdose_y = np.linspace(rtdose_origin[1], rtdose_origin[1] + rtdose_spacing[1]*(rtdose_size[1] - 1), rtdose_size[1])
rtdose_z = np.linspace(rtdose_origin[2], rtdose_origin[2] + rtdose_spacing[2]*(rtdose_size[2] - 1), rtdose_size[2])

_, axs = plt.subplots(1,3, figsize=(15,5))

axs[0].set_title('axial')
axs[0].imshow(rtdose[:,:,130], 
              extent=[rtdose_y[0], rtdose_y[-1], rtdose_x[-1], rtdose_x[0]])

axs[1].set_title('coronal')
axs[1].imshow(rtdose[150,:,:].T,
              extent=[rtdose_y[0], rtdose_y[-1], rtdose_z[-1], rtdose_z[0]])
axs[1].invert_yaxis()

axs[2].set_title('sagittal')
axs[2].imshow(rtdose[:,70,:].T, 
              extent=[rtdose_x[0], rtdose_x[-1], rtdose_z[-1], rtdose_z[0]])
axs[2].invert_yaxis()

plt.show()

In [ ]:
rtdose.shape

In [111]:
''' save nii '''

rtdose_affine = np.array([
    [rtdose_spacing[0], 0, 0, rtdose_origin[0]],
    [0, rtdose_spacing[1], 0, rtdose_origin[1]],
    [0, 0, rtdose_spacing[2], rtdose_origin[2]],
    [0, 0, 0, 1]
])

rtdose_nii = nib.Nifti1Image(rtdose, rtdose_affine)

# nib.save(rtdose_nii, '../data/rtdose.nii')

In [ ]:
''' read nii '''
rtdose_data = nib.load('../data/rtdose.nii')

rtdose_info = rtdose_data.header
rtdose_affine = rtdose_data.affine
rtdose = rtdose_data.get_fdata()

print(rtdose_info)

In [ ]:
''' plot nii '''
rtdose_origin = rtdose_affine[:3,3]
rtdose_spacing = rtdose_info.get_zooms()
rtdose_size = rtdose_info['dim'][1:4]
rtdose_size = rtdose_size[::-1] # python(z y x) : 317 143 267 ----> dcm/nii(x y z)

rtdose_x = np.linspace(rtdose_origin[0], rtdose_origin[0] + rtdose_spacing[0]*(rtdose_size[0] - 1), rtdose_size[0])
rtdose_y = np.linspace(rtdose_origin[1], rtdose_origin[1] + rtdose_spacing[1]*(rtdose_size[1] - 1), rtdose_size[1])
rtdose_z = np.linspace(rtdose_origin[2], rtdose_origin[2] + rtdose_spacing[2]*(rtdose_size[2] - 1), rtdose_size[2])

_, axs = plt.subplots(1,3, figsize=(15,5))

axs[0].set_title('axial')
axs[0].imshow(rtdose[:,:,130], 
              extent=[rtdose_y[0], rtdose_y[-1], rtdose_x[-1], rtdose_x[0]])

axs[1].set_title('coronal')
axs[1].imshow(rtdose[150,:,:].T,
              extent=[rtdose_y[0], rtdose_y[-1], rtdose_z[-1], rtdose_z[0]])
axs[1].invert_yaxis()

axs[2].set_title('sagittal')
axs[2].imshow(rtdose[:,70,:].T, 
              extent=[rtdose_x[0], rtdose_x[-1], rtdose_z[-1], rtdose_z[0]])
axs[2].invert_yaxis()

plt.show()

In [11]:
# rtst_info = dicominfo(RTstFile, 'UseVRHeuristic', false);
# contour = dicomContours(rtst_info);